In [1]:
from pathlib import Path

In [2]:
def parse(file):
    return [int(n) for n in Path('./data/'+file).read_text().split('\n')]

## Part I
Find two numbers that sum to 2020.

In [3]:
def solve_PartI(file):
    report = parse(file)
    go = True
    while go:
        entry1 = report.pop()
        for entry2 in report:
            if entry1 + entry2 == 2020:
                out = entry1*entry2
                print(entry1, '*', entry2, '=', out)
                go = False
    return out
solve_PartI('AoC20_01t.txt')

299 * 1721 = 514579


514579

In [4]:
solve_PartI('AoC20_01.txt')

403 * 1617 = 651651


651651

## Part II
Find three numbers that sum to 2020
- [itertools.combinations(iterable, r)](https://docs.python.org/3/library/itertools.html#itertools.combinations): Return r length subsequences of elements from the input iterable.

Roughly equivalent to:
```py
def combinations(iterable, r):
    # combinations('ABCD', 2) --> AB AC AD BC BD CD
    # combinations(range(4), 3) --> 012 013 023 123
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)
```

In [5]:
from itertools import combinations

In [6]:
for file in ['AoC20_01t.txt', 'AoC20_01.txt']:
    print(file, ':')    
    report = parse(file)
    out2 = [x for x in combinations(report,2) if x[0]+x[1] == 2020][0]
    print('\t', out2[0], '*', out2[1], '=', out2[0]*out2[1])
    out3 = [x for x in combinations(report,3) if x[0]+x[1]+x[2] == 2020][0]
    print('\t', out3[0], '*', out3[1], '*', out3[2], '=', out3[0]*out3[1]*out3[2])

AoC20_01t.txt :
	 1721 * 299 = 514579
	 979 * 366 * 675 = 241861950
AoC20_01.txt :
	 1617 * 403 = 651651
	 1144 * 372 * 504 = 214486272


## Learn an Algo for Combinations 
Wiki : [Combination](https://en.wikipedia.org/wiki/Combination)

**[Why does python use 'else' after for and while loops?](https://stackoverflow.com/questions/9979970/why-does-python-use-else-after-for-and-while-loops)** (StackOverflow)

A common construct is to run a loop until something is found and then to break out of the loop. The problem is that if I break out of the loop or the loop ends I need to determine which case happened. One method is to create a flag or store variable that will let me do a second test to see how the loop was exited.

For example assume that I need to search through a list and process each item until a flag item is found and then stop processing. If the flag item is missing then an exception needs to be raised.

Using the Python for...else construct you have

```py
for i in mylist:
    if i == theflag:
        break
    process(i)
else:
    raise ValueError("List argument missing terminal flag.")
```

Compare this to a method that does not use this syntactic sugar:

```py
flagfound = False
for i in mylist:
    if i == theflag:
        flagfound = True
        break
    process(i)

if not flagfound:
    raise ValueError("List argument missing terminal flag.")
```
In the first case the raise is bound tightly to the for loop it works with. In the second the binding is not as strong and errors may be introduced during maintenance.

In [34]:
def combinations_debug(iterable, r):
    # combinations('ABCD', 2) --> AB AC AD BC BD CD
    # combinations(range(4), 3) --> 012 013 023 123
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    print(f"yield {tuple(pool[i] for i in indices)}, where indices are {indices} & enter while loop:")
    while True:
        print(f"\tEnter next yielding cycle and check positions backwards (i.e. {list(reversed(range(r)))})")
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                print(f"\t- index on position {i} ({indices[i]}) doesn't equal {i+n-r} (i+n-r: {i}+{n-r}), so break checking and process it!")
                break
            print(f"\t- index on position {i} ({indices[i]}) equals to {i+n-r} (i+n-r: {i}+{n-r}), check previous position ...")
        else:
            print("End yielding loop!")
            return
        print(f"\tIncrement value on position {i} ({indices[i]}<-{indices[i]+1})")
        indices[i] += 1
        rest_range = list(range(i+1, r))
        if rest_range == []:
            print("\t> Skip processing the rest of this combination.")
        else:
            print(f"\tNow restart sequence on positions in range {rest_range}:")
        for j in range(i+1, r):
            print(f"\t\tindices[{j}] = indices[{j-1}] ({indices[j-1]}) + 1 ({indices[j]}<-{indices[j-1]+1})")
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)
        print(f"\tyield {tuple(pool[i] for i in indices)}, where indices are {indices}\n")
list(combinations_debug('ABCD',2))

yield ('A', 'B'), where indices are [0, 1] & enter while loop:
	Enter next yielding loop and check positions backwards (i.e. [1, 0])
	- index on position 1 (1) doesn't equal 3 (i+n-r: 1+2), so break checking and process it!
	Increment value on position 1 (1<-2)
	> Skip processing the rest of this combination.
	yield ('A', 'C'), where indices are [0, 2]

	Enter next yielding loop and check positions backwards (i.e. [1, 0])
	- index on position 1 (2) doesn't equal 3 (i+n-r: 1+2), so break checking and process it!
	Increment value on position 1 (2<-3)
	> Skip processing the rest of this combination.
	yield ('A', 'D'), where indices are [0, 3]

	Enter next yielding loop and check positions backwards (i.e. [1, 0])
	- index on position 1 (3) equals to 3 (i+n-r: 1+2), check previous position ...
	- index on position 0 (0) doesn't equal 2 (i+n-r: 0+2), so break checking and process it!
	Increment value on position 0 (0<-1)
	Now restart sequence on positions in range [1]:
		indices[1] = indices

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]

In [35]:
list(combinations_debug(range(5),3))

yield (0, 1, 2), where indices are [0, 1, 2] & enter while loop:
	Enter next yielding loop and check positions backwards (i.e. [2, 1, 0])
	- index on position 2 (2) doesn't equal 4 (i+n-r: 2+2), so break checking and process it!
	Increment value on position 2 (2<-3)
	> Skip processing the rest of this combination.
	yield (0, 1, 3), where indices are [0, 1, 3]

	Enter next yielding loop and check positions backwards (i.e. [2, 1, 0])
	- index on position 2 (3) doesn't equal 4 (i+n-r: 2+2), so break checking and process it!
	Increment value on position 2 (3<-4)
	> Skip processing the rest of this combination.
	yield (0, 1, 4), where indices are [0, 1, 4]

	Enter next yielding loop and check positions backwards (i.e. [2, 1, 0])
	- index on position 2 (4) equals to 4 (i+n-r: 2+2), check previous position ...
	- index on position 1 (1) doesn't equal 3 (i+n-r: 1+2), so break checking and process it!
	Increment value on position 1 (1<-2)
	Now restart sequence on positions in range [2]:
		indic

[(0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 2, 3),
 (0, 2, 4),
 (0, 3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4)]